In [ ]:
!pip install transformers torch pandas numpy

Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: torch 2.9.0+cpu
Uninstalling torch-2.9.0+cpu:
  Successfully uninstalled torch-2.9.0+cpu
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 178.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 701.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 865.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB

In [ ]:
# basic sentiment analysis pipeline

import numpy as np
import pandas as pd
data = pd.read_csv("emotion_dataset.csv")
text = data["text"][:10]
text = text.to_list()
text

['you were broken-hearted and the world was too',
 'are you afraid, because I’m terrified',
 'happiness hit her like a train on a track',
 'and I am freaking out in the middle of the street',
 'I can see no way, I can see no way',
 'I was never as good as I always thought I was',
 'this is a song for a scribbled out name and my love keeps writing again and again',
 'Jesus Christ, it hurts',
 "I'm miles away, he's on my mind",
 'Well, did you miss me? Walk on water just to kiss me?']

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta", device_map="auto", dtype=torch.bfloat16)
vocab =  ["sadness", "love", "fear", "pain", "happiness", "anger", "despair"]
res = []
for i in text:
  messages = [
      {"role": "system", "content": f"""You are an LLM that analyzes sentences based on the following labels: {vocab}.
      You read in a sentence and reply ONLY with the following labels: {vocab} NOT with POSITIVE/NEGATIVE.
      You also give a confidence score. Classify EACH sentence independently and return one label per sentence in the same order. I repeat, ONLY ONE label per sentence.
      Stop IMMEDIATELY generating after one label/Confidence pair.
      The format you will be outputting this is as follows:"""},
      {"role": "user", "content": "I feel awful and exhausted."},
      {"role": "assistant", "content": "label: sadness | Confidence: 0.92"},
      {"role": "user", "content": "I can't believe this worked so well!"},
      {"role": "assistant", "content": "label: happiness | Confidence: 0.89"},
      {"role": "user", "content": i},
  ]

  inputs = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      device="cuda",
      return_tensors="pt")

  outputs = model.generate(
      inputs,
      max_new_tokens=30,
      do_sample=False,
      temperature=0.0,
      eos_token_id=tokenizer.eos_token_id,
  )

  # Decode only the newly generated tokens
  generated_text = tokenizer.decode(outputs[0, inputs.shape[1]:], skip_special_tokens=True)
  res.append(generated_text)

print(res)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:2534: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by cal

['label: sadness | Confidence: 0.98', 'label: fear | Confidence: 0.95', 'label: happiness | Confidence: 0.98', 'label: fear | Confidence: 0.95', 'label: despair | Confidence: 0.98', 'label: despair | Confidence: 0.95', 'label: love | Confidence: 0.98', 'label: pain | Confidence: 0.98', 'label: love | Confidence: 0.91', 'label: love | Confidence: 0.98']


In [ ]:
import re

parsed_results = []
for entry in res:
    # Find the last occurrence of <|assistant|>
    last_assistant_start_index = entry.rfind("<|assistant|>")
    if last_assistant_start_index != -1:
        # Get the substring starting from the last <|assistant|>
        last_assistant_part = entry[last_assistant_start_index:]

        # Extract the content after the last <|assistant|> token
        # Use (.*?) to allow for empty content
        match_assistant = re.search(r"<|assistant|>(.*?)(?:</s>|$)", last_assistant_part, re.DOTALL)
        if match_assistant:
            assistant_response_content = match_assistant.group(1)
            assistant_response = assistant_response_content.strip() if assistant_response_content is not None else ""

            # Extract label and confidence from the assistant's response, making it robust to whitespace
            match_label_confidence = re.search(r"label:\s*(\w+)\s*\|\s*Confidence:\s*([0-9.]+)", assistant_response)
            if match_label_confidence:
                label = match_label_confidence.group(1)
                confidence = float(match_label_confidence.group(2))
                parsed_results.append({"label": label, "confidence": confidence})
            else:
                # Handle cases where label and confidence are not perfectly matched
                # This can happen if the model output is malformed or truncated
                # Try to extract a label if possible, otherwise assign N/A
                if "label:" in assistant_response:
                    try:
                        potential_label = assistant_response.split("label:")[1].split("|")[0].strip()
                        parsed_results.append({"label": potential_label, "confidence": "N/A"})
                    except IndexError:
                        parsed_results.append({"label": "N/A", "confidence": "N/A"})
                else:
                    parsed_results.append({"label": "N/A", "confidence": "N/A"}) # Default if no label pattern found
        else:
            # This should ideally not happen if last_assistant_start_index was found,
            # but for safety, if the regex on last_assistant_part fails
            parsed_results.append({"label": "N/A", "confidence": "N/A"})
    else:
        # This block is executed if <|assistant|> is not found at all in the entry.
        parsed_results.append({"label": "N/A", "confidence": "N/A"})

results_df = pd.DataFrame(parsed_results)

# Add the original text to the DataFrame if 'text' variable is available and matches length
if 'text' in globals() and len(text) == len(results_df):
    results_df['original_text'] = text

results_df.to_csv("sentiment_results.csv", index=False)
print("Results saved to sentiment_results.csv")
print(results_df.head())

Results saved to sentiment_results.csv
  label confidence                                      original_text
0   N/A        N/A      you were broken-hearted and the world was too
1   N/A        N/A              are you afraid, because I’m terrified
2   N/A        N/A          happiness hit her like a train on a track
3   N/A        N/A  and I am freaking out in the middle of the street
4   N/A        N/A                 I can see no way, I can see no way
